<a href="https://colab.research.google.com/github/meguia/IRma/blob/main/IRMA_Remoto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esta es una Notebook de Colab para utilizar IRMA (Impulse Response Measurement and Analysis) de forma remota sin interfaz grafica.

Posee una funcionalidad similar excepto el manejo de la placa de audio ya que al correr en una maquina remota no puede acceder a la interfaz.

Esta notebook puede ser util en los casos en los que se reproduzca y grabe el barrido de frecuencias en dispositivos diferentes.

En ese caso el proceso es el siguiente:

1. Ejecutar la seccion "Preparacion maquina remota y Acceso al Drive" que instala los paquetes necesarios en la maquina remota y solicita el acceso a un Drive que se utilizara para almacenar y guardar datos. Tener en cuenta que al correr de forma remota todo lo que se ejecute se pierde al cerrarse la sesion o luego de un tiempo de espera, por eso es necesario trabajar vinculado a una carpeta de Drive. En este caso se asume que el usuario tiene creada una carpeta en Drive llamada "IRma" en el Drive, si no existe la crea.
2. Ejecutar la seccion "Generacion de Sweep y Creacion de Sesion" completando con los datos relevantes para el proyecto. Si el sweep no esta generado lo crea en la carpeta IRma junto con el filtro inverso. Dentro de IRma va a crear una subcarpeta con el nombre de la Sesion.
3. Descargar el sweep generado desde el Drive y reproducirlo y grabarlo localmente. Es muy importante que: (a) el sample rate usado para generar el sweep sea el mismo utilizado para reproducir y grabar (b) dar un nombre consistente a las grabaciones para luego poder indentificarlas durante el analisis, lo ideal es que comienzen con "rec_" asi despues la notebook puede renombrarlas como "ir_". Por ejemplo si se graba desde el centro de la sala un parlante ubicado en la izquierda se puede llamar a la grabacion "rec_Left_Centro_Toma1.wav" y asi luego la respuesta impulso sera "ir_Left_Centro_Toma1.wav". Todas las grabaciones deben almacenarse en la carpeta Subcarpeta con el nombre de la sesion dentro de "IRma" de Drive. Por ejemplo si nuestra sesion se llama "Pasillo" todos los wavs grabados deben ir dentro de "IRma/Pasillo" y no afuera.
4. Volver a la notebook y volver a Ejecutar la seccion "Preparacion de maquina remota y Acceso al Drive" porque probablemente haya caducado la sesion.
5. Correr la seccion "Extraccion de Respuestas Impulso" e indicar en la celda correspondiente el nombre de la sesion. Las celdas siguientes van a leer todas las grabaciones en la subcarpeta con el nombre de la sesion y van a extrer las respuestas impulso almacenandolas en la misma. Si se da la opcion de graficar se muestran todas los graficos de las respuestas impulso.
6. Finalmente correr la ultima seccion para analizar los parametros acusticos, respuesta en frecuencia, etc.

# 1. Preparacion de la Maquina Remota y Acceso al Drive

In [2]:
# @title
!pip install   --force-reinstall "git+https://github.com/meguia/irma"

  Cloning https://github.com/meguia/irma to c:\users\camilo\appdata\local\temp\pip-req-build-zp2rvle5
  Resolved https://github.com/meguia/irma to commit 9b8a933a0297da93d8a10c84ee842f0e3fbc4700
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for irma: filename=irma-0.1.3-py3-none-any.whl size=41426 sha256=fa458b5fc8bc1a420bceca14a650d8547f01f7c897464d9ee8ec03a8b21cc061
  Stored in directory: C:\Users\Camilo\AppData\Local\Temp\pip-ephem-wheel-cache-as37tr11\wheels\ec\5c\11\a61defe7beb6bb3bb78c30b5d11323c5e7ce7abf29c63c0d15
Successfully built irma
  Attempting uninstall: irma
    Found existing installation: irma 0.1.3
    Uninstalling irma-0.1.3:
      Successfully uninstalled irma-0.1.3


  Running command git clone --filter=blob:none --quiet https://github.com/meguia/irma 'C:\Users\Camilo\AppData\Local\Temp\pip-req-build-zp2rvle5'


In [3]:
# @title
import os
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
from irma.session_nosd import RecordingSession
from irma.generate import sweep
from irma.process import make_filterbank,ir_extract,ir_list_to_multichannel
from irma.room import paracoustic
from irma.display import *
from IPython.display import Audio, display

In [4]:
# Primera celda donde hay que meter mano
# global
fs = 96000 # frecuencia de sampleo
# dar nombre de sesion
sesion = "Chapa"

In [5]:
# @title
spath = '/content/drive/MyDrive/IRma/'+sesion+'/'
if not os.path.exists(spath):
  print("creando directorio "+spath)
  os.makedirs(spath)
else:
  print("trabajando en directorio "+spath)

creando directorio /content/drive/MyDrive/IRma/Chapa/


# 2. Generacion de Sweep y Creacion de Sesion

Correr solo la primera vez

In [6]:
# parametros del sweep
dur = 20 # duracion del barrido
fmin = 40.0 # frecuencia inicial
fmax = 24000.0 # frecuencia final
sweepfile = f"sweep_{fs//1000}k_{int(dur)}s_{int(fmin)}_{int(fmax//1000)}k"
sweep(dur,fmin,fmax,spath+sweepfile,fs,post=1.0);

Warning RMS pedido mayor al RMS de corte que es -3.46 dB 
Sweep RMS = -3.46 dB 


AttributeError: module 'scipy.signal' has no attribute 'hann'

In [9]:
parlantes = ["L","R","centro"]
mics = ["pub1","pub2"]
tipo = "Mono"
s = RecordingSession(sesion,parlantes,mics,inchan=[],outchan=[],rtype=tipo,
                     sampling_rate=fs,sweepfile=spath+sweepfile,recordingpath=spath)
s.save_metadata(spath+sesion)

# 3. Extraccion de Respuestas Impulso

Correr solo una vez luego de la grabacion

In [125]:
sesion = "Caverna"
spath = '/content/drive/MyDrive/IRma/'+sesion+'/'
s =  RecordingSession.load_metadata(spath+sesion)

In [126]:
# CORRER SOLO Si se grabaron sweeps
s.load_rec_from_dir(spath);

Extracting ---> L3D1M1-190604_1757_1 using sr = 96000
Extracting ---> L3D2M6-190604_1830_1 using sr = 96000
Extracting ---> L3D3M4-190604_1845 using sr = 96000


In [ ]:
# CORRER SOLO Si se grabaron impulsos dirtectos (descomentar quitar el numeral del principio)
# s.load_rec_from_dir(spath,apply_filter=False);

# 4. Visualizacion y escucha de la Respuesta Impulso
Si es multicanal grafica solo el primer canal

In [127]:
# @title
ir_pick1 = widgets.Dropdown(
    options=s.recordings_tuples(),
    value=0,
    description='Resp. Imp.:',
)
ir_pick2 = widgets.Dropdown(
    options=s.recordings_tuples(),
    value=0,
    description='Espectro:',
)
ir_pick3 = widgets.Dropdown(
    options=s.recordings_tuples(),
    value=0,
    description='Parametros:',
)
def dropdown_event_handler(change):
    with widgets.Output():
      clear_output()

ir_pick1.observe(dropdown_event_handler, names='value')
ir_pick2.observe(dropdown_event_handler, names='value')

In [128]:
# @title
@widgets.interact(ir_number=ir_pick1)
def plot(ir_number):
  print("...")
  ir = s.load_ir(ir_number)
  if ir.ndim == 1:
    ir = ir[:,np.newaxis]
  display(Audio(ir[:,0],rate=s.sampling_rate))
  ir_plot(ir[:,0],labels=[s.recordings[ir_number]['filename']])
  plt.show()

interactive(children=(Dropdown(description='Resp. Imp.:', options=(('L3D1M1-190604_1757_1', 0), ('L3D2M6-19060…

In [130]:
# @title
@widgets.interact(ir_number=ir_pick2)
def splot(ir_number):
  ir = s.load_ir(ir_number)
  if ir.ndim == 1:
    ir = ir[:,np.newaxis]
  spectrum_plot(ir[:,0],fmax=1000, fs=fs,labels=[s.recordings[ir_number]['filename']])
  plt.show()

interactive(children=(Dropdown(description='Espectro:', options=(('L3D1M1-190604_1757_1', 0), ('L3D2M6-190604_…

# 5. Parametros Acusticos

In [ ]:
# correr solo la primera vez
make_filterbank(fmin=62.5,noct=9,bwoct=1,fs=s.sampling_rate,bankname=spath+'fbank48k_9_1',show=True)

Banco de filtros generado: 9 octavas,1 bandas/octava,desde 62.5 Hz,Almacenada en archivo /content/drive/MyDrive/IRma/Caverna/fbank48k_9_1


In [132]:
# @title
ir_pick = widgets.Dropdown(
    options=s.recordings_tuples(),
    value=0,
    description='Respuesta Impulso:',
)
@widgets.interact(ir_number=ir_pick3)
def parameters(ir_number):
  ir = s.load_ir(ir_number)
  pars = paracoustic(ir, method='RT15',bankname=spath+'fbank48k_9_1')
  pars_print(pars)

interactive(children=(Dropdown(description='Parametros:', options=(('L3D1M1-190604_1757_1', 0), ('L3D2M6-19060…

In [133]:
# @title
key_pick = widgets.Dropdown(
    options=['RT15','EDT','C50','C80','TS','DRR','SNR'],
    value='RT15',
    description='Parametro:',
)

In [144]:
# @title
ir_pick = widgets.SelectMultiple(
    options=s.recordings_tuples(),
    value=[0],
    description='Respuesta Impulso:',
)
display(key_pick)
@widgets.interact(ir_selected=ir_pick)
def plot_compared(ir_selected):
  ir_list = s.load_ir_list(ir_selected,ftype='npy')
  ir_stacked = ir_list_to_multichannel(ir_list)
  params = paracoustic(ir_stacked, method="RT15",bankname=spath+'fbank48k_9_1')
  pars_compared_axes(params, key_pick.value,labels=[s.recordings[n]['filename'] for n in list(ir_pick.value)])
  plt.show()


Dropdown(description='Parametro:', options=('RT15', 'EDT', 'C50', 'C80', 'TS', 'DRR', 'SNR'), value='RT15')

interactive(children=(SelectMultiple(description='Respuesta Impulso:', index=(0,), options=(('L3D1M1-190604_17…